In [1]:
import pandas as pd

In [2]:
# CSV 경로
# csv_path = '/workspace/Data_Augumentation/Albumentation/label/train/albumentation_train.csv'    # Albumentation
# csv_path = '/workspace/Data_Augumentation/out_box_masking/label/train/mask_boxout.csv'          # masking out of box
csv_path = '/workspace/Data_Augumentation/segmentation_face/label/train/face_only_train.csv'    # crop face

# 새로운 CSV 저장할 경로
# output_csv_path = '/workspace/data_for_yolo/yolo_album.csv'
# output_csv_path = '/workspace/data_for_yolo/yolo_mask_boxout.csv'
output_csv_path = '/workspace/data_for_yolo/yolo_face_crop.csv'

# 저장할 이미지 경로
# output_image_folder = '/workspace/data_for_yolo/Albumentation/'
# output_image_folder = '/workspace/data_for_yolo/mask_boxout/'
output_image_folder = '/workspace/data_for_yolo/face_crop/'


#### YOLO 이미지 생성

In [3]:
import pandas as pd
import cv2
import os

# CSV 파일 읽기
df = pd.read_csv(csv_path)

# 리사이즈할 크기
resize_width, resize_height = 640, 640

# 이미지가 저장될 폴더가 없다면 생성
os.makedirs(output_image_folder, exist_ok=True)

# 이미지 리사이즈 및 바운딩 박스 조정
resized_data = []
for index, row in df.iterrows():
    if index % 100 == 0:
        print(f'{index} / {len(df)}')
    # 원본 이미지 경로
    image_path = row['path'] + row['filename']
    
    # 이미지를 로드하고 크기 조정
    img = cv2.imread(image_path)
    original_height, original_width = img.shape[:2]
    resized_img = cv2.resize(img, (resize_width, resize_height))
    
    # 바운딩 박스 비율에 맞춰 조정
    scale_x = resize_width / original_width
    scale_y = resize_height / original_height
    new_minX = row['minX'] * scale_x
    new_maxX = row['maxX'] * scale_x
    new_minY = row['minY'] * scale_y
    new_maxY = row['maxY'] * scale_y
    
    # 새로운 이미지 저장
    new_image_name = f"yolo_{row['filename']}"
    new_image_path = os.path.join(output_image_folder, new_image_name)
    cv2.imwrite(new_image_path, resized_img)
    
    # 조정된 데이터를 새로운 리스트에 추가
    resized_data.append({
        'filename': new_image_name,
        'maxX': new_maxX,
        'maxY': new_maxY,
        'minX': new_minX,
        'minY': new_minY,
        'faceExp_uploader': row['faceExp_uploader'],
        'path': output_image_folder
    })

# 새로운 CSV 파일로 저장
resized_df = pd.DataFrame(resized_data)
resized_df.to_csv(output_csv_path, index=False)

print(f"Resized images and updated CSV saved to {output_image_folder} and {output_csv_path}.")


0 / 5993
100 / 5993
200 / 5993
300 / 5993
400 / 5993
500 / 5993
600 / 5993
700 / 5993
800 / 5993
900 / 5993
1000 / 5993
1100 / 5993
1200 / 5993
1300 / 5993
1400 / 5993
1500 / 5993
1600 / 5993
1700 / 5993
1800 / 5993
1900 / 5993
2000 / 5993
2100 / 5993
2200 / 5993
2300 / 5993
2400 / 5993
2500 / 5993
2600 / 5993
2700 / 5993
2800 / 5993
2900 / 5993
3000 / 5993
3100 / 5993
3200 / 5993
3300 / 5993
3400 / 5993
3500 / 5993
3600 / 5993
3700 / 5993
3800 / 5993
3900 / 5993
4000 / 5993
4100 / 5993
4200 / 5993
4300 / 5993
4400 / 5993
4500 / 5993
4600 / 5993
4700 / 5993
4800 / 5993
4900 / 5993
5000 / 5993
5100 / 5993
5200 / 5993
5300 / 5993
5400 / 5993
5500 / 5993
5600 / 5993
5700 / 5993
5800 / 5993
5900 / 5993
Resized images and updated CSV saved to /workspace/data_for_yolo/face_crop/ and /workspace/data_for_yolo/yolo_face_crop.csv.


#### YOLO 라벨 파일 생성

In [4]:
# CSV 파일 로드
df = pd.read_csv(output_csv_path)

# 감정 클래스 매핑
emotion_classes = {'기쁨': 0, '분노': 1, '슬픔': 2, '당황': 3}

# 데이터 변환 함수 (CSV -> YOLO 형식)
def create_yolo_format(df, save_dir, image_width, image_height):
    os.makedirs(save_dir, exist_ok=True)
    for idx, row in df.iterrows():
        img_path = row['path']
        filename = row['filename']
        minX, maxX, minY, maxY = row['minX'], row['maxX'], row['minY'], row['maxY']
        emotion = row['faceExp_uploader']
        
        # 좌표 정규화: YOLO는 (0, 1) 범위로 정규화해야 한다
        x_center = (minX + maxX) / 2 / image_width
        y_center = (minY + maxY) / 2 / image_height
        width = (maxX - minX) / image_width
        height = (maxY - minY) / image_height
        
        # 범위 체크: 좌표 값이 0과 1 사이인지 확인
        if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
            print(f"Skipping {filename}: Bounding box coordinates out of bounds.")
            continue
        
        # 감정 정보 라벨 추가
        emotion_label = emotion_classes.get(emotion, -1)
        if emotion_label == -1:
            print(f"Skipping {filename}: Invalid emotion class.")
            continue
        
        # YOLO 형식의 라벨 생성 
        label = f"{emotion_label} {x_center} {y_center} {width} {height}"  
        
        # 라벨 파일 생성
        label_file = os.path.join(save_dir, f"{filename.split('.')[0]}.txt")
        with open(label_file, 'w') as f:
            f.write(label)  # 얼굴 바운딩 박스
        

# 이미지 크기 (예시: 640x480)
image_width = 640
image_height = 640

# YOLO 형식 데이터 생성
create_yolo_format(df, output_image_folder, image_width, image_height)

